<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#библиотеки" data-toc-modified-id="библиотеки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>библиотеки</a></span></li><li><span><a href="#данные" data-toc-modified-id="данные-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>данные</a></span></li><li><span><a href="#признаки" data-toc-modified-id="признаки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>признаки</a></span></li><li><span><a href="#обучение-предсказание" data-toc-modified-id="обучение-предсказание-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>обучение предсказание</a></span></li><li><span><a href="#тест-2-патологии-на-1-снимке" data-toc-modified-id="тест-2-патологии-на-1-снимке-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>тест 2 патологии на 1 снимке</a></span></li></ul></div>

In [1]:
# # Сохранение модели в файл
# joblib.dump(model, 'catboost_model_mm_v4.pkl')

# # Для загрузки модели :
# # loaded_model = joblib.load('catboost_model_v4.pkl')

In [2]:
# ВАЖНО!
# обучать нужно на снимках где только 1 патология

## библиотеки

In [3]:
# !pip install catboost

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import optuna
import joblib

# индикатор выполнения
from tqdm import tqdm 

# Установка уровня логирования для Optuna на WARNING, чтобы подавить информационные сообщения
import logging
optuna.logging.set_verbosity(optuna.logging.WARNING)

## данные

In [4]:
path = r"E:\gnat\2\mm2.xlsx"
df = pd.read_excel(path)
df

,ID,Проекция,Сторона,"left, мм","top, мм","right, мм",condyle_width\n(ширина мыщелка),condyle_height\nвысота мыщелка),art_fossa_width\n(ширина ямки),art_fossa_depth\n(глубина ямки),Патология
0,100,Сагиттальная PR2,Левая,2.3,2.2,1.9,5.3,2.2,9.0,3.2,compression
1,102,Сагиттальная PR2,Левая,2.5,2.2,2.5,7.0,2.8,12.1,4.0,compression
2,108,Сагиттальная PR2,Левая,2.7,1.9,0.9,4.8,3.0,7.8,4.0,compression
3,109,Сагиттальная PR2,Левая,2.7,2.8,1.0,5.9,5.4,9.1,4.6,compression
4,101,Сагиттальная PR2,Левая,2.3,4.0,1.9,7.2,2.1,11.1,4.8,distal
5,105,Сагиттальная PR2,Левая,1.7,2.8,1.6,6.5,2.3,9.1,4.3,distal
6,106,Сагиттальная PR2,Левая,3.3,3.2,1.8,4.9,1.4,9.9,3.1,distal
7,118,Сагиттальная PR2,Левая,3.5,2.3,1.2,4.1,1.6,9.0,3.1,distal
8,107,Сагиттальная PR2,Левая,1.1,4.0,2.3,6.5,1.9,9.7,5.0,mesial
9,117,Сагиттальная PR2,Левая,1.5,2.3,2.8,5.4,1.4,9.4,2.9,mesial


## признаки

In [5]:
# Создание соотношения измерений
df['left, мм / top, мм']   = df['left, мм']  / df['top, мм']
df['left, мм / right, мм'] = df['left, мм']  / df['right, мм']
df['top, мм / left, мм']   = df['top, мм']   / df['left, мм']
df['top, мм / right, мм']  = df['top, мм']   / df['right, мм']
df['right, мм / left, мм'] = df['right, мм'] / df['left, мм']
df['right, мм / top, мм']  = df['right, мм'] / df['top, мм']

In [6]:
columns_to_drop = ["ID", "Проекция", "Сторона"]
# columns_to_drop = ["ID", "Проекция", "Сторона",
#                   "condyle_width\n(ширина мыщелка)",
#                    "condyle_height\nвысота мыщелка)",
#                    "art_fossa_width\n(ширина ямки)",
#                    "art_fossa_depth\n(глубина ямки)"]
df = df.drop(columns=columns_to_drop)
df

,"left, мм","top, мм","right, мм",condyle_width\n(ширина мыщелка),condyle_height\nвысота мыщелка),art_fossa_width\n(ширина ямки),art_fossa_depth\n(глубина ямки),Патология,"left, мм / top, мм","left, мм / right, мм","top, мм / left, мм","top, мм / right, мм","right, мм / left, мм","right, мм / top, мм"
0,2.3,2.2,1.9,5.3,2.2,9.0,3.2,compression,1.045455,1.210526,0.956522,1.157895,0.826087,0.863636
1,2.5,2.2,2.5,7.0,2.8,12.1,4.0,compression,1.136364,1.000000,0.880000,0.880000,1.000000,1.136364
2,2.7,1.9,0.9,4.8,3.0,7.8,4.0,compression,1.421053,3.000000,0.703704,2.111111,0.333333,0.473684
3,2.7,2.8,1.0,5.9,5.4,9.1,4.6,compression,0.964286,2.700000,1.037037,2.800000,0.370370,0.357143
4,2.3,4.0,1.9,7.2,2.1,11.1,4.8,distal,0.575000,1.210526,1.739130,2.105263,0.826087,0.475000
5,1.7,2.8,1.6,6.5,2.3,9.1,4.3,distal,0.607143,1.062500,1.647059,1.750000,0.941176,0.571429
6,3.3,3.2,1.8,4.9,1.4,9.9,3.1,distal,1.031250,1.833333,0.969697,1.777778,0.545455,0.562500
7,3.5,2.3,1.2,4.1,1.6,9.0,3.1,distal,1.521739,2.916667,0.657143,1.916667,0.342857,0.521739
8,1.1,4.0,2.3,6.5,1.9,9.7,5.0,mesial,0.275000,0.478261,3.636364,1.739130,2.090909,0.575000
9,1.5,2.3,2.8,5.4,1.4,9.4,2.9,mesial,0.652174,0.535714,1.533333,0.821429,1.866667,1.217391


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   left, мм                        20 non-null     float64
 1   top, мм                         20 non-null     float64
 2   right, мм                       20 non-null     float64
 3   condyle_width
(ширина мыщелка)  20 non-null     float64
 4   condyle_height
высота мыщелка)  20 non-null     float64
 5   art_fossa_width
(ширина ямки)   20 non-null     float64
 6   art_fossa_depth
(глубина ямки)  20 non-null     float64
 7   Патология                       20 non-null     object 
 8   left, мм / top, мм              20 non-null     float64
 9   left, мм / right, мм            20 non-null     float64
 10  top, мм / left, мм              20 non-null     float64
 11  top, мм / right, мм             20 non-null     float64
 12  right, мм / left, мм            20 non

## обучение предсказание

In [8]:
%%time

def objective_with_progress(trial):
    return objective(trial)

# Установка глобального рандом сида
random_seed = 1

# Разделение на признаки и целевую переменную
X = df.drop('Патология', axis=1)
y = df['Патология']

# Отделение по одному объекту каждого класса для тестовой и валидационной выборок
test_indices = df.groupby('Патология').apply(lambda x: x.sample(1, random_state=random_seed)).index.get_level_values(1)
remaining_indices = df.index.difference(test_indices)
validation_indices = df.loc[remaining_indices].groupby('Патология').apply(lambda x: x.sample(1, random_state=random_seed)).index.get_level_values(1)
train_indices = remaining_indices.difference(validation_indices)

# Создание обучающей, тестовой и валидационной выборок
X_train, X_test, X_val = X.loc[train_indices], X.loc[test_indices], X.loc[validation_indices]
y_train, y_test, y_val = y.loc[train_indices], y.loc[test_indices], y.loc[validation_indices]

# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Объединение в один датасет для обучения
X_train_full = np.concatenate([X_train, X_val], axis=0)
y_train_full = np.concatenate([y_train, y_val], axis=0)

def objective(trial):
    # Гиперпараметры для оптимизации
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_strength': trial.suggest_int('random_strength', 1, 20),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0)
    }

    # Создание и обучение модели
    model = CatBoostClassifier(**param, random_seed=random_seed, verbose=0)
    model.fit(X_train_full, y_train_full)

    # Оценка модели
    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')  # Использование F1-меры
    return f1

# def objective(trial):
#     # Гиперпараметры для оптимизации
#     param = {
#         'iterations': trial.suggest_int('iterations', 100, 1000),
#         'depth': trial.suggest_int('depth', 4, 10),
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
#         'random_strength': trial.suggest_int('random_strength', 1, 20),
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
#         'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10.0)
#     }

#     # Создание и обучение модели
#     model = CatBoostClassifier(**param, random_seed=random_seed, verbose=0)
#     model.fit(X_train_full, y_train_full)

#     # Оценка модели
#     preds = model.predict_proba(X_val)
#     auc = roc_auc_score(y_val, preds, multi_class='ovo')  # Использование AUC-ROC для многоклассовой классификации
#     return auc
# # ovo (one-vs-one): AUC рассчитывается для каждой пары классов и усредняется.
# # ovr (one-vs-rest): AUC рассчитывается для каждого класса против всех остальных и усредняется.

# Создание и выполнение Optuna исследования с индикатором выполнения
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=random_seed))
with tqdm(total=100, desc="Processing", unit="trial") as pbar:
    def callback(study, trial):
        pbar.update(1)
    study.optimize(objective_with_progress, n_trials=100, callbacks=[callback])

# Лучшие параметры
best_params = study.best_params
print('Лучшие параметры:', best_params)

# Обучение модели с лучшими параметрами
model = CatBoostClassifier(**best_params, random_seed=random_seed, verbose=0)
model.fit(X_train_full, y_train_full)

# Предсказания на тестовом наборе с лучшей моделью
predictions = model.predict(X_test)
predictions

Processing: 100%|██████████| 100/100 [01:04<00:00,  1.56trial/s]


Лучшие параметры: {'iterations': 475, 'depth': 9, 'learning_rate': 0.010033168697030017, 'random_strength': 7, 'bagging_temperature': 0.14675589081711304, 'od_type': 'Iter', 'l2_leaf_reg': 3.462151663160047}
CPU times: total: 6min 8s
Wall time: 1min 4s


array([['compression'],
       ['compression'],
       ['distraction'],
       ['mesial'],
       ['norm']], dtype=object)

In [9]:
# Получение предсказанных вероятностей для тестового набора данных
predictions_proba = model.predict_proba(X_test)
# Получение названий классов из модели
class_names = model.classes_

# Отсортированные по убыванию вероятности предсказания для каждого объекта
sorted_formatted_predictions = []
for proba in predictions_proba:
    # Сортировка пар (класс, вероятность) по убыванию вероятности
    sorted_proba = sorted(zip(class_names, proba), key=lambda x: x[1], reverse=True)
    # Форматирование и добавление в итоговый список
    prediction_str = ", ".join([f"{class_name}: {prob*100:.2f}%" for class_name, prob in sorted_proba])
    sorted_formatted_predictions.append(prediction_str)
sorted_formatted_predictions

# ['compression: 25.82%, distal: 25.05%, distraction: 18.20%, norm: 16.15%, mesial: 14.77%',
#  'compression: 29.70%, distal: 24.22%, norm: 15.99%, distraction: 15.46%, mesial: 14.64%',
#  'distraction: 27.43%, mesial: 21.99%, norm: 19.39%, distal: 17.07%, compression: 14.12%',
#  'mesial: 26.60%, norm: 21.39%, distraction: 20.13%, distal: 17.91%, compression: 13.97%',
#  'norm: 24.67%, distraction: 20.30%, distal: 19.72%, mesial: 18.35%, compression: 16.97%']

['compression: 25.82%, distal: 25.05%, distraction: 18.20%, norm: 16.15%, mesial: 14.77%',
 'compression: 29.70%, distal: 24.22%, norm: 15.99%, distraction: 15.46%, mesial: 14.64%',
 'distraction: 27.43%, mesial: 21.99%, norm: 19.39%, distal: 17.07%, compression: 14.12%',
 'mesial: 26.60%, norm: 21.39%, distraction: 20.13%, distal: 17.91%, compression: 13.97%',
 'norm: 24.67%, distraction: 20.30%, distal: 19.72%, mesial: 18.35%, compression: 16.97%']

In [10]:
y_test

3     compression
7          distal
19    distraction
11         mesial
15           norm
Name: Патология, dtype: object

## тест 2 патологии на 1 снимке

In [11]:
path = r"E:\gnat\2\mm2_test_2patology.xlsx"
df = pd.read_excel(path)

# Создание соотношения измерений
df['left, мм / top, мм']   = df['left, мм']  / df['top, мм']
df['left, мм / right, мм'] = df['left, мм']  / df['right, мм']
df['top, мм / left, мм']   = df['top, мм']   / df['left, мм']
df['top, мм / right, мм']  = df['top, мм']   / df['right, мм']
df['right, мм / left, мм'] = df['right, мм'] / df['left, мм']
df['right, мм / top, мм']  = df['right, мм'] / df['top, мм']

columns_to_drop = ["ID", "Проекция", "Сторона"]
# columns_to_drop = ["ID", "Проекция", "Сторона",
#                   "condyle_width\n(ширина мыщелка)",
#                    "condyle_height\nвысота мыщелка)",
#                    "art_fossa_width\n(ширина ямки)",
#                    "art_fossa_depth\n(глубина ямки)"]
df = df.drop(columns=columns_to_drop)

# Разделение на признаки и целевую переменную
X = df.drop('Патология', axis=1)
y = df['Патология']

# Нормализация данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Предсказания на тестовом наборе с лучшей моделью
predictions = model.predict(X)
predictions

# Получение предсказанных вероятностей для тестового набора данных
predictions_proba = model.predict_proba(X)
# Получение названий классов из модели
class_names = model.classes_

# Отсортированные по убыванию вероятности предсказания для каждого объекта
sorted_formatted_predictions = []
for proba in predictions_proba:
    # Сортировка пар (класс, вероятность) по убыванию вероятности
    sorted_proba = sorted(zip(class_names, proba), key=lambda x: x[1], reverse=True)
    # Форматирование и добавление в итоговый список
    prediction_str = ", ".join([f"{class_name}: {prob*100:.2f}%" for class_name, prob in sorted_proba])
    sorted_formatted_predictions.append(prediction_str)
sorted_formatted_predictions

['distraction: 24.96%, mesial: 24.09%, norm: 18.50%, distal: 16.40%, compression: 16.05%',
 'compression: 27.58%, distal: 22.19%, norm: 17.81%, distraction: 16.59%, mesial: 15.84%']

In [12]:
df['Патология']

0    mesial + distraction
1    compression + mesial
Name: Патология, dtype: object